In [232]:
import os
import glob
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import pydicom
import cv2
#from joblib import Parallel, delayed

In [233]:
#cropping by metadata
#input: img - dcm file
#output: cropped img file
"""def old_metacrop(img):
    print(type(img))
    temp = (img.get_item((0x0018,0x6011)).value)
    print(temp)
    x0 = temp[0x00186018].value
    x1 = temp[0x0018601c].value
    y0 = temp[0x0018601a].value
    y1 = temp[0x0018601e].value
    return img.pixel_array[x0:x1, y0:y1]"""
def metacrop(file):
    for key in file.dir():
        value = getattr(file, key, "")
        if(key == "SequenceOfUltrasoundRegions"):
            value = value[0]
            break
    x0, x1, y0, y1 = None, None, None, None
    for key in value.dir():
        if key == "RegionLocationMinX0":
            x0 = getattr(value, key, "")
        if key == "RegionLocationMaxX1":
            x1 = getattr(value, key, "")
        if key == "RegionLocationMinY0":
            y0 = getattr(value, key, "")
        if key == "RegionLocationMaxY1":
            y1 = getattr(value, key, "")
    return file.pixel_array[y0:y1+1, x0:x1+1]

In [234]:
ROOT = "/data3/wv2019/data/PLIC_MILANO_DICOM"
ROOT_OUT = "/data3/wv2019/data/processed/PLIC_MILANO_DICOM"

In [235]:
# define callbacks
def person_names_callback(dataset, data_element):
    if data_element.VR == "PN":
        data_element.value = "anonymous"
        
def curves_callback(dataset, data_element):
    if data_element.tag.group & 0xFF00 == 0x5000:
        del dataset[data_element.tag]

t2tag = 'PatientBirthDate'

In [ ]:
### TODO
# - consider also Doppler mode
# - extend to the whole set of images
# - check whether the anonymized DICOM file differs from the original (except for the removed sensitive data)
# - parallelize code

In [ ]:
data_len = len(glob.glob(os.path.join(ROOT, "*", "GEMS_IMG", "*", "*", "*", "*")))

In [249]:
!find "/data3/wv2019/data/PLIC_MILANO_DICOM/" | grep "BANFITI_79422"

find: unknown predicate `-r'


In [ ]:
# directory tree:
# ROOT
# - PLIC<something>_DISCO<N>
#   - GEMS_IMG
#     - <YEAR>_<MONTH>
#       - <NN>
#         - <SOMEDIR>
#           - DICOMs

# organize DICOMs by patient ID
# start with a subset of imgs
for fn in glob.glob(os.path.join(ROOT, "*", "GEMS_IMG", "*", "*", "*", "*"), recursive=True):
    ds = pydicom.read_file(fn)

    # clear private data
    ds.walk(person_names_callback)
    ds.walk(curves_callback)
    # address/sanitize patient IDs (remove spaces and non-alphanum characters)
    ds.data_element('PatientID').value = re.sub(r'\W+', '', ds.data_element('PatientID').value.strip().replace(" ", "_"))
    # type 2 tags
    if t2tag in ds:
        ds.data_element(t2tag).value = ''

    patientID = ds.PatientID
    dicomName = os.path.basename(fn).strip().replace(" ", "_")

    out_dir = os.path.join(ROOT_OUT, str(patientID))
    os.makedirs(out_dir, exist_ok=True)
    out_dicom = os.path.join(out_dir, f"{dicomName}.dcm")
    
    desc = str(ds)
    for line in desc.split("\n"):
        if "#DopplerFreq" in line:
            out_dir = os.path.join(ROOT_OUT, "DOPPLER", str(patientID))
            os.makedirs(out_dir, exist_ok=True)
            out_dicom = os.path.join(out_dir, f"{dicomName}.dcm")
            break
    #out_png = os.path.join(out_dir, f"conv_{dicomName}.png")
    #plt.imshow(ds.pixel_array)
    #plt.show()
    
    #print(fn)
    
    # write DICOM Standard compliant file
    if not os.path.isfile(out_dicom):
        pydicom.filewriter.write_file(out_dicom, ds, write_like_original=False)
    # write PNG for non-doppler images
    # NOTE: not needed at first, let's keep also Doppler mode
    #if len(ds.SequenceOfUltrasoundRegions) == 1 :
    #    if not os.path.isfile(out_png):
    #        cv2.imwrite(out_png, ds.pixel_array)


In [ ]:
# Metacrop function test
#pdata = metacrop(ds)

#ds1.PixelData = pdata.tobytes()
#ds1.Rows, ds1.Columns, _ = pdata.shape